## Step 1: Create Lookup Color - Image Table

In [1]:
# define path to directory containing the images that will be used for the mosaic

In [6]:
import subprocess
from PIL import Image
import numpy as np
path = "/Users/francesco/work/all_emojis/"
emoji_dict = {}
ls_lines = [line for line in subprocess.check_output(['ls',path]).splitlines() if "png" in line]

In [7]:
## define distance function to be used later

In [8]:
def distance(x, y):
    return (x[0] - y[0])**2+(x[1] - y[1])**2+(x[2] - y[2])**2

In [11]:
for emoji_path in ls_lines:
    # find average color value for each image.
    # 160x160 is the size of the emojis that I used. This shouldn't be hardcoded
    mean_value = tuple(np.mean(np.reshape(np.asarray(Image.open(path + emoji_path).rotate(90).convert('RGB')), (160*160,3)), axis =0))
    emoji_dict[mean_value] = emoji_path

In [ ]:
colors_range  = range(0,255,4)
colorDict = {}
for r in colors_range:
    for g in colors_range:
        for b in colors_range:
            colorDict[(r,g,b)] = emoji_dict[min(emoji_dict.keys(), key=lambda x: distance(x, (r, g, b)))]

## Step 2 -Upload to a S3 Bucket your images - this way you can use cloud hosting services such as Heroku

## Step 3 - Create Image to IOString Table(Used to load faster images from S3)

In [14]:
import io, urllib2
path_s3 =  "https://s3-eu-west-1.amazonaws.com/emojimosaic/all_emoji/"
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

In [ ]:
def returnIOString(img, hdr):
    try:
        req = urllib2.Request(path_s3+img, headers=hdr)
        return io.BytesIO(urllib2.urlopen(req).read())
    except urllib2.HTTPError, e:
        return e.fp.read()   

In [ ]:
imgs = list(set(emoji_dict.values()))

In [ ]:
dict_url_image = dict([(img, returnIOString(img,hdr)) for img in imgs])

## Step 4 - Save color-image lookup table and image - IOString table

In [15]:
import pickle

In [16]:
def save_obj(obj, name ):
    with open(name+'.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
save_obj(dict_url_image, "dict_url_image" )

In [ ]:
save_obj(colorDict, "colorDict" )